In [ ]:
#Installing required Packages
!pip install selenium
!pip install webdriver_manager


In [1]:
#Importing packages
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By

import re
import time
import os
import pandas as pd

In [2]:
#Setting chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
browser = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.80/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\anvin\.wdm\drivers\chromedriver\win32\98.0.4758.80]
C:\Users\anvin\AppData\Local\Temp/ipykernel_11152/2433240595.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chrome_options)
C:\Users\anvin\AppData\Local\Temp/ipykernel_11152/2433240595.py:4: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chrome_options)


In [3]:
wait = WebDriverWait(browser, 5)

In [4]:
smartphone_url='https://www.amazon.in/s?i=electronics&bbn=1805560031&rh=n%3A1805560031%2Cp_89%3ARedmi%7CSamsung&dc&fs=true&qid=1644240816&rnid=3837712031&ref=sr_nr_p_89_2'
#smartphone_url='https://www.amazon.in/s?i=electronics&bbn=1805560031&rh=n%3A1805560031%2Cp_89%3ATecno&dc&fs=true&qid=1642520011&rnid=3837712031&ref=sr_nr_p_89_6'

In [5]:
def get_lastpage_number():
    try:
        xpath_pagination = '//span[@class="s-pagination-strip"]'
        last_page_obj = wait.until(ec.visibility_of_element_located((By.XPATH, xpath_pagination)))
        last_page_text = last_page_obj.text
        last_page_match_obj = re.search(r'\.\s*(\d+)\s*Next', last_page_text)
        if last_page_match_obj != None:
            last_page = last_page_match_obj.group(1).strip()
        else:
            last_page_match_obj = re.search(r'(\d)\s*Next', last_page_text)
            last_page = last_page_match_obj.group(1).strip() if last_page_match_obj != None else ''
    except Exception as e:
        try:
            xpath_pagination = '//ul[@class="a-pagination"]'
            last_page_obj = wait.until(ec.visibility_of_element_located((By.XPATH, xpath_pagination)))
            last_page_text = last_page_obj.text
            last_page_match_obj = re.search(r'(\d+)\nNext', last_page_text)
            last_page = last_page_match_obj.group(1).strip() if last_page_match_obj != None else ''        
        except Exception as e:
            print(e)
            last_page=''
    return last_page

In [12]:
def change_pin_code(zip_code):
    time.sleep(2)
    browser.find_element_by_id('nav-global-location-data-modal-action').click()
    time.sleep(2)
    input_field = browser.find_element_by_id('GLUXZipUpdateInput')
    input_field.clear()
    input_field.send_keys(zip_code)
    apply_button_xpath = '//input[@aria-labelledby="GLUXZipUpdate-announce"]'
    apply_button = browser.find_element_by_id('GLUXZipUpdate')
    apply_button.click()
    time.sleep(3)
    browser.get(smartphone_url)

In [13]:
def get_product_links(zip_code):
    browser.get(smartphone_url)
    change_pin_code(zip_code)
    last_page_number = get_lastpage_number()
    print("Number of pages to scrape : ", last_page_number)
    xpath_next_btn_pagination_1 = '//li[@class="a-last"]'
    xpath_next_btn_pagination_2 = '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]'

    last_page_number = int(last_page_number) + 1
    #looping through pagination links
    xpath_product_link = '//a[@class="a-link-normal s-no-outline"]'
    product_links = []
    for i in range(1, last_page_number):
        if i == 1:
            browser.get(smartphone_url)

        links = wait.until(ec.visibility_of_all_elements_located((By.XPATH, xpath_product_link)))
        for link in links:
            product_links.append(link.get_attribute('href'))

        if i == last_page_number-1:
            break
        time.sleep(3)
        try:
            next_btn = wait.until(ec.visibility_of_element_located((By.XPATH, xpath_next_btn_pagination_1)))
            next_btn.click()
        except Exception as e:
            next_btn = wait.until(ec.visibility_of_element_located((By.XPATH, xpath_next_btn_pagination_2)))
            next_btn.click()
    print('Number of product links found : ',len(product_links))   
    return product_links

In [18]:
def get_product_details(product_url):
    browser.get(product_url)
    try:
        prod_info = wait.until(ec.visibility_of_element_located((By.ID,'centerCol')))
        prod_info = prod_info.text
        prod_info = os.linesep.join([s for s in prod_info.splitlines() if s])
    except Exception as e:
        prod_info = ''
    
    try:
        prod_tech_info = wait.until(ec.visibility_of_element_located((By.ID,'productDetails_techSpec_section_1')))
        prod_tech_info = prod_tech_info.text
    except Exception as e:
        prod_tech_info = ''
    
    
    try:
        prod_additional_info = wait.until(ec.visibility_of_element_located((By.ID,'productDetails_detailBullets_sections1')))
        prod_additional_info = prod_additional_info.text
    except Exception as e:
        prod_additional_info = ''
    
    try:
        prod_buy_info = wait.until(ec.visibility_of_element_located((By.ID,'desktop_qualifiedBuyBox')))
        prod_buy_info = prod_buy_info.text
    except Exception as e:
        prod_buy_info = ''
    
    prod_data = prod_info +'\n\n'+ prod_tech_info +'\n\n'+ prod_additional_info +'\n\n'+ prod_buy_info
    return prod_data
    

In [19]:
prod_df_tmp = pd.DataFrame()
def extract_pruduct_info(prod_data, zip_code):
    prod_mrp_price_obj = re.search(r'm\.r\.p\.\:\s*\n(.*)', prod_data, re.IGNORECASE)
    prod_mrp_price = prod_mrp_price_obj.group(1) if prod_mrp_price_obj != None else ''
    
    prod_sale_price_obj = re.search(r'price:\s*\n(.*)', prod_data, re.IGNORECASE)
    prod_sale_price = prod_sale_price_obj.group(1) if prod_sale_price_obj != None else ''
    
    deal_of_the_day_obj = re.search(r'deal\s+of\s+the\s+day\:\s*\n(.*)', prod_data, re.IGNORECASE)
    deal_of_the_day = deal_of_the_day_obj.group(1) if deal_of_the_day_obj != None else ''
    
    deal_end_time_obj = re.search(r'ends\s+in\s+(.*)', prod_data, re.IGNORECASE)
    deal_end_time = deal_end_time_obj.group(1) if deal_end_time_obj != None else ''
    
    your_savings_obj = re.search(r'you\s+save\:\s*\n(.*)', prod_data, re.IGNORECASE)
    your_savings = your_savings_obj.group(1) if your_savings_obj != None else ''
    
    emi_available_obj = re.search(r'emi\s+starts\s+at', prod_data, re.IGNORECASE)
    emi_available = 'yes' if emi_available_obj != None else ''
    
    nocost_emi_available_obj = re.search(r'no\s+cost\s+emi\s+available', prod_data, re.IGNORECASE)
    nocost_emi_available = 'yes' if nocost_emi_available_obj != None else ''
    
    product_model_obj = re.search(r'model\s+name\s+(.*)', prod_data, re.IGNORECASE)
    product_model = product_model_obj.group(1) if product_model_obj != None else ''
    
    wireless_carrier_obj = re.search(r'wireless\s+carrier\s+(.*)', prod_data, re.IGNORECASE)
    wireless_carrier = wireless_carrier_obj.group(1) if wireless_carrier_obj != None else ''
    
    product_brand_obj = re.search(r'brand\s+(.*)', prod_data, re.IGNORECASE)
    product_brand = product_brand_obj.group(1) if product_brand_obj != None else ''
    
    product_memory_storage_obj = re.search(r'memory\s+storage\s+capacity\s+(.*)', prod_data, re.IGNORECASE)
    product_memory_storage = product_memory_storage_obj.group(1) if product_memory_storage_obj != None else ''
    
    product_os_obj = re.search(r'\nos\s+(.*)', prod_data, re.IGNORECASE)
    product_os = product_os_obj.group(1) if product_os_obj != None else ''
    
    product_ram_obj = re.search(r'\nram\s+(.*)', prod_data, re.IGNORECASE)
    product_ram = product_ram_obj.group(1) if product_ram_obj != None else ''
    
    product_colour_obj = re.search(r'\ncolour\s+(.*)', prod_data, re.IGNORECASE)
    product_colour = product_colour_obj.group(1) if product_colour_obj != None else ''

    product_manufacturer_obj = re.search(r'\nmanufacturer\s+(.*)', prod_data, re.IGNORECASE)
    product_manufacturer = product_manufacturer_obj.group(1) if product_manufacturer_obj != None else ''
    
    product_country_of_origin_obj = re.search(r'\ncountry\s+of\s+origin\s+(.*)', prod_data, re.IGNORECASE)
    product_country_of_origin = product_country_of_origin_obj.group(1) if product_country_of_origin_obj != None else ''    
    
    prod_condition = ''
    if 'renewed' in prod_data.lower() or 'refurbished' in prod_data.lower():
        prod_condition = 'Old'
    else:
        prod_condition = 'New'
    
    prod_df_tmp['Country'] = ['India']
    prod_df_tmp['Currency'] = ['INR']
    prod_df_tmp['prod_mrp_price'] = [prod_mrp_price.strip()]
    prod_df_tmp['prod_sale_price'] = [prod_sale_price.strip()]
    prod_df_tmp['deal_of_the_day'] = [deal_of_the_day.strip()]
    prod_df_tmp['deal_end_time'] = [deal_end_time.strip()]
    prod_df_tmp['your_savings'] = [your_savings.strip()] 
    prod_df_tmp['emi_available'] = [emi_available.strip()]
    prod_df_tmp['nocost_emi_available'] = [nocost_emi_available.strip()]
    prod_df_tmp['product_model'] = [product_model.strip()]
    prod_df_tmp['wireless_carrier'] = [wireless_carrier.strip()]
    prod_df_tmp['product_brand'] = [product_brand.strip()]
    prod_df_tmp['product_memory_storage'] = [product_memory_storage.strip()]
    prod_df_tmp['product_os'] = [product_os.strip()]
    prod_df_tmp['product_ram'] = [product_ram.strip()]
    prod_df_tmp['product_colour'] = [product_colour.strip()]
    prod_df_tmp['product_condition'] = [prod_condition]
    prod_df_tmp['product_manufacturer'] = [product_manufacturer.strip()]
    prod_df_tmp['product_country_of_origin'] = [product_country_of_origin.strip()]
    prod_df_tmp['zip_code']=[zip_code]
    prod_df_tmp['tmp_prod_data']=[prod_data]
    return prod_df_tmp


In [20]:
def clean_product_name(p_name):
    p_name = str(p_name)
    p_name = re.sub(r'\(.*\)', '', p_name)
    return p_name

def clean_price(p_price):
    p_price = str(p_price)
    p_price = re.sub(r'\,'
                     '|\.00'
                     '|\₹'
                     '|\(.*\)', '', p_price)
    return p_price

In [21]:
prod_df = pd.DataFrame(columns = ['Country', 'Currency', 'prod_mrp_price',  'prod_sale_price', 'deal_of_the_day'
                                  , 'deal_end_time', 'your_savings', 'emi_available', 'nocost_emi_available', 'product_model'
                                  , 'wireless_carrier', 'product_brand','product_memory_storage', 'product_os', 'product_ram'
                                  , 'product_colour', 'product_manufacturer', 'product_country_of_origin', 'zip_code', 'tmp_prod_data'])

zip_codes = ['110006', '530068'] # Add
zip codes here

for zip_code in zip_codes:
    product_links = get_product_links(zip_code)
    for p_link in product_links:
        prod_data = get_product_details(p_link)
        prod_df_tmp = extract_pruduct_info(prod_data, zip_code)
        prod_df = prod_df.append(prod_df_tmp)


C:\Users\anvin\AppData\Local\Temp/ipykernel_11152/3245080623.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  browser.find_element_by_id('nav-global-location-data-modal-action').click()
C:\Users\anvin\AppData\Local\Temp/ipykernel_11152/3245080623.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  input_field = browser.find_element_by_id('GLUXZipUpdateInput')
C:\Users\anvin\AppData\Local\Temp/ipykernel_11152/3245080623.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  apply_button = browser.find_element_by_id('GLUXZipUpdate')


Number of pages to scrape :  13
Number of product links found :  293
Number of pages to scrape :  14
Number of product links found :  311


In [22]:
prod_df['prod_mrp_price'] = prod_df['prod_mrp_price'].apply(clean_price)
prod_df['prod_sale_price'] = prod_df['prod_sale_price'].apply(clean_price)
prod_df['deal_of_the_day'] = prod_df['deal_of_the_day'].apply(clean_price)
prod_df['your_savings'] = prod_df['your_savings'].apply(clean_price)
prod_df['product_model'] = prod_df['product_model'].apply(clean_product_name)

In [23]:
prod_df.head()

,Country,Currency,prod_mrp_price,prod_sale_price,deal_of_the_day,deal_end_time,your_savings,emi_available,nocost_emi_available,product_model,...,product_brand,product_memory_storage,product_os,product_ram,product_colour,product_manufacturer,product_country_of_origin,zip_code,tmp_prod_data,product_condition
0,India,INR,25990,22999,,,2991,yes,yes,Galaxy M32 5G,...,Samsung,128 GB,Android 11,8 GB,Sky Blue,Samsung India pvt Ltd,India,110006,"Samsung Galaxy M32 5G (Sky Blue, 8GB RAM, 128G...",New
0,India,INR,36999,26999,,,10000,yes,yes,Samsung Galaxy M52 5G,...,Samsung,128 GB,Android 11,8 GB,Blazing Black,Samsung India pvt Ltd,India,110006,"Samsung Galaxy M52 5G (Blazing Black, 8GB RAM,...",New
0,India,INR,12999,10499,,,2500,yes,yes,Galaxy M12,...,Samsung,64 GB,Android 11,4 GB,Blue,Samsung India pvt Ltd,India,110006,"Samsung Galaxy M12 (Blue,4GB RAM, 64GB Storage...",New
0,India,INR,14979,12999,,,1980,yes,yes,Redmi 10 Prime,...,Redmi,64 GB,"MIUI 12.5, Android MIUI 12.5",4 GB,Astral White,Redmi,India,110006,Redmi 10 Prime (Astral White 4GB RAM 64GB ROM ...,New
0,India,INR,15499,12499,,,3000,yes,yes,Galaxy M12,...,Samsung,128 GB,Android 11,6 GB,Blue,Samsung India pvt Ltd,India,110006,"Samsung Galaxy M12 (Blue,6GB RAM, 128GB Storag...",New


In [24]:
prod_df.to_excel('sample_output.xlsx', index=False)